In [1]:
import json
import numpy as np
from pandas.io.json import json_normalize

import os
import glob as gb
import tensorflow as tf

from sklearn.model_selection import train_test_split
import keras 
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM , Flatten , CuDNNLSTM , GRU, Conv1D, MaxPooling1D, Bidirectional
from keras.layers import Dropout
import keras

from keras import callbacks
from keras.callbacks import  CSVLogger


# Model Evaluations
from sklearn import metrics
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import KFold

In [2]:
def load_file(filepath,i):
    with open(filepath) as f:
        data = json.load(f)
    imu = data['imu']['data']
    emg = data['emg']['data']
    emg = np.array(emg)
    imu = np.array(imu)
    imu_gyr = np.array([(e['gyroscope']) for e in imu])
    imu_acc = np.array([(e['acceleration']) for e in imu])
    imu_orn = np.array([(e['orientation']) for e in imu])
    #timestamp = [i]
    #timestamp = np.repeat(timestamp, 400, axis=None)
    #timestamp= timestamp.reshape(400,1)
    dataset = tf.concat([emg, imu_gyr, imu_acc, imu_orn], axis=1, name='concat')
    dataset = np.array(dataset)
    return dataset

In [3]:
data_path = '../input/emgimu/An-EMG-and-IMU-Dataset-for-the-Italian-Sign-Language-Alphabet-master/Dataset/'

In [4]:
# For Files Data
X = []
y = []
i=0
for folder in  os.listdir(data_path) : 
    j=0
    files = gb.glob(pathname= str( data_path  + folder + '/*.json'))
    for file in files: 
        data = load_file(file,j)
        X.append(data)
        y.append(i)
        j+=1
    i+=1

print(f'we have {len(X)} items in X ')

2022-10-26 19:54:45.896287: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 19:54:45.897219: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 19:54:46.246027: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 19:54:46.246989: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 19:54:46.247734: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

we have 780 items in X 


In [5]:
X = np.array(X)
y = np.array(y)

In [6]:
from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=20)

In [7]:
print('X shape is : ' , X.shape)
print('y shape is : ' , y.shape)

X shape is :  (780, 400, 18)
y shape is :  (780,)


In [8]:
mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 
           'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 
           'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}

In [9]:
def getcode(n) : 
    for x , y in mapping.items() : 
        if n == y : 
            return x

In [10]:
X = X.reshape(len(X), X.shape[1],18)
y = to_categorical(y)

In [11]:
print(X.shape)

(780, 400, 18)


# Building Model with files

In [12]:
kfold = KFold(n_splits=5, shuffle=True)

In [13]:
acc_per_fold = []
loss_per_fold = []
fold_no = 1
for train, test in kfold.split(X, y):

  # Model architecture
    model = Sequential()
    model.add(Conv1D(128, 3, activation='relu', input_shape =(400,18)))
    model.add(MaxPooling1D(3))
    model.add(Conv1D(64, 3, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Bidirectional(CuDNNLSTM(units=128, return_sequences=True)))
    model.add(Bidirectional(CuDNNLSTM(units=64, return_sequences=True)))
    model.add(Dropout(rate=0.3))
    model.add(Flatten())
    model.add(Dense(26,activation='softmax'))


    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#     es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
#     checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="./checkpoint-{epoch:02d}.hdf5",
#                                                       verbose=1, save_best_only=True,
#                                                       monitor='val_acc',mode='max')
#     csv_logger = CSVLogger('training_set_iranalysis3.csv',separator=',', append=False)




    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    
    history = model.fit(X[train], y[train], 
                        batch_size=128, epochs=40)

    
    scores = model.evaluate(X[test], y[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; \
    {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...


2022-10-26 19:54:59.984995: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/40


2022-10-26 19:55:04.562730: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


5/5 [==============================] - 17s 59ms/step - loss: 3.1490 - accuracy: 0.1042
Epoch 2/40
5/5 [==============================] - 0s 23ms/step - loss: 2.1009 - accuracy: 0.4519
Epoch 3/40
5/5 [==============================] - 0s 23ms/step - loss: 1.2162 - accuracy: 0.6763
Epoch 4/40
5/5 [==============================] - 0s 23ms/step - loss: 0.7083 - accuracy: 0.7949
Epoch 5/40
5/5 [==============================] - 0s 22ms/step - loss: 0.4663 - accuracy: 0.8413
Epoch 6/40
5/5 [==============================] - 0s 22ms/step - loss: 0.3916 - accuracy: 0.8750
Epoch 7/40
5/5 [==============================] - 0s 22ms/step - loss: 0.2724 - accuracy: 0.9199
Epoch 8/40
5/5 [==============================] - 0s 23ms/step - loss: 0.1950 - accuracy: 0.9407
Epoch 9/40
5/5 [==============================] - 0s 23ms/step - loss: 0.1564 - accuracy: 0.9519
Epoch 10/40
5/5 [==============================] - 0s 22ms/step - loss: 0.1149 - accuracy: 0.9551
Epoch 11/40
5/5 [======================

In [14]:
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.22620975971221924 - Accuracy: 92.94871687889099%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.0808258205652237 - Accuracy: 96.79487347602844%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.14851294457912445 - Accuracy: 95.51281929016113%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.17298579216003418 - Accuracy: 93.58974099159241%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.12071198970079422 - Accuracy: 96.79487347602844%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 95.12820482254028 (+- 1.6012829697084006)
> Loss: 0.14984926134347915
-----------------------